# Produccion eléctrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a la produccion eléctrica de energias renovables y no renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto en principio es prácticamente inmediato, también vamos a centrarnos en la produccion de energia hidroelectrica para poder medir cuanto tiempo tarda en afectar a este tipo de energía las precipitaciones..

En base a esto tenemos varias cuestiones que queremos despejar:
- Que relación hay entre precipitaciones y el aumento de la generación de energia eléctrica hidráulica.
- Que relación hay entre horas de sol, temperatura y el aumento de la generación de energia eléctrica sólar.
- Que relación hay entre viento y el aumento de la generación de energia eléctrica eólica, esta relación esta condicionada por la temperatura, percipitaciones o horas de sol.
- Cuantos días tarda en aumentar la generación eléctrica de fuentes de energía renovables en función de los fenomenos metereológicos.
- Dados una prediccion meteorologica que valores de generación eléctrica tendremos para una fecha determinada.



Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Debido a la falta de datos a nivel diario de REE por provincia o comunidades autonomas, voy a centrar el analisis a nivel de sistema eléctrico (Peninsula, Baleares, Canarias ,Ceuta y Melilla).

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Requisitos para la ejecución del notebook:

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request.

Además las versiones de cada libreria utilizada en este proyecto son:

El modelo ha utilizar al tratarse de una prediccion númerica y no de obtener una etiqueta, sera una regresión.
Para poder llevar a cabo esa regresion se han pasado los valores de fechas a númericos, y se ha creado una columna por cada tipo de energia, creando varias variables objetivo.



In [1]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

pd.options.display.max_columns=None


###  Lectura datos AEMET

Para la lectura de datos metereólogicos, vamos a utilizar la libreria aemet, de la que utilizaremos los metodos de las clases Aemet y Estacion para obtener los datos a nivel diario de cada estación meterologica para un rago de fechas, de las provincias que nos interesan para el estudio.

In [2]:
# Leemos la clave de la API de AEMET desde un fichero ubicado en el directorio ../API que este notebook

with open('../API/API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()


# Obtenemos el json de estaciones de mediciones de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE);

In [3]:
from tqdm import tqdm_notebook as tqdm

# Definimos funciones que vamos a utilizar para leer los datos de AEMET
def estaciones_prov (lista_estaciones):
    '''Dada una lista de provincias y un json de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    
    lista_id=[]
    print('Reading list of id of weather stations...')
    for estacion in tqdm(lista_estaciones):
            lista_id.append(estacion['indicativo'])
            
    return lista_id



def lectura_diaria_lista(date_ini,date_end,lista_estaciones):
    '''Dado una lista de id de estaciones de aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016':
        date_ini="2016-01-01T00:00:00UTC"
        
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    
    print("Reading AEMET data from %s to %s ..." % (date_ini,date_end))
    
    for element in lista_estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;

In [4]:
date_ini="2016-01-01T00:00:00UTC"
date_end="2021-02-28T00:00:00UTC"

year_ini=int(date_ini[0:4])
year_fin=int(date_end[0:4])




In [5]:
id_estaciones=estaciones_prov(info_estaciones)

df_weather=pd.DataFrame()
for i in tqdm(np.arange(year_ini,year_fin+1)):
    if i==year_ini:
        inicio=date_ini
        fin=str(i)+"-12-31T00:00:00UTC"
        
    elif i==year_fin:
        inicio=str(i)+"-01-01T00:00:00UTC"
        fin=date_end
        
    else:
        inicio=str(i)+"-01-01T00:00:00UTC"
        fin=str(i)+"-12-31T00:00:00UTC"
        
    df_weather=df_weather.append(pd.DataFrame(lectura_diaria_lista(inicio,fin,id_estaciones),dtype=str))
    
print('Finish reading AEMET date from %s to %s' % (df_weather['fecha'].min(),df_weather['fecha'].max()) )

Reading list of id of weather stations...


Reading AEMET data from 2016-01-01T00:00:00UTC to 2016-12-31T00:00:00UTC ...
Reading AEMET data from 2017-01-01T00:00:00UTC to 2017-12-31T00:00:00UTC ...
Reading AEMET data from 2018-01-01T00:00:00UTC to 2018-12-31T00:00:00UTC ...
Reading AEMET data from 2019-01-01T00:00:00UTC to 2019-12-31T00:00:00UTC ...
Reading AEMET data from 2020-01-01T00:00:00UTC to 2020-12-31T00:00:00UTC ...
Reading AEMET data from 2021-01-01T00:00:00UTC to 2021-02-28T00:00:00UTC ...

Finish reading AEMET date from 2016-01-01 to 2021-02-24


### Limpieza datos AEMET

In [6]:
''' Creo una funcion que pase los indicadores a float 
y rellene los valores vacios por la media de las estaciones de esa provincia para ese dia'''

def rellena_nulos_provincia(df,cols):
    # Defino un DataFrame vacio para acumular el resultado
    df_all=pd.DataFrame()
    
    # Hago un bucle para cada provincia del DataFrame de entrada
    for prov in df['provincia'].unique():
        df_prov=df[df['provincia']==prov].copy()
        
        #Para cada elemento de las columnas que nos interesan reemplazo , por ., paso a numerico y relleno los nulos por la media de la provincia
        for element in cols:
            df_prov[element]=df_prov[element].str.replace(',', '.')
            df_prov[element]=pd.to_numeric(df_prov[element],errors='coerce')
            df_prov[element].fillna(df_prov[element].mean(skipna=True),inplace=True)
        df_all=df_all.append(df_prov)
    return df_all


In [7]:
df_aux=df_weather.copy()

In [8]:
# Elimino las columnas que no me interesan
df_weather.drop(columns=['altitud','horaPresMax','horaPresMin','horaracha','dir','horatmin','horatmax','racha'],inplace=True)

# Limpio de nulos la muestra
df_weather=rellena_nulos_provincia(df_weather,df_weather.columns[5:])


In [11]:
electric_systems={
    'STA. CRUZ DE TENERIFE':'canarias',
    'LAS PALMAS':'canarias',
    'ILLES BALEARS':'baleares',
    'CEUTA':'ceuta',
    'MELILLA':'melilla'}

In [13]:
# Ahora que ya tienen el mismo peso cada provincia, agrupo por sistema eléctrico.

df_weather['system']=[electric_systems[l.upper()]  
                      if l.upper() in electric_systems.keys() else 'peninsular' 
                      for l in df_weather['provincia'] ]

In [9]:
df_weather

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2016-01-01,0252D,ARENYS DE MAR,BARCELONA,74,"11,2","0,2","7,8",06:20,"14,7",12:40,24,"1,7","7,2",14:00,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,0252D,ARENYS DE MAR,BARCELONA,74,"11,8","0,0","7,5",23:40,"16,1",13:10,24,"1,1","9,7",11:40,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,0252D,ARENYS DE MAR,BARCELONA,74,"10,0","0,0","5,8",05:50,"14,1",13:20,24,"1,9","9,2",16:00,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,0252D,ARENYS DE MAR,BARCELONA,74,"11,6","0,9","8,0",22:50,"15,3",10:10,32,"0,8","8,9",02:40,NaN,NaN,NaN,NaN,NaN
4,2016-01-05,0252D,ARENYS DE MAR,BARCELONA,74,"9,9","0,1","5,6",23:59,"14,2",12:30,32,"2,5","8,6",17:30,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13119,2021-02-20,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"18,8","0,1","14,2",02:45,"23,4",14:00,99,"11,7","22,2",Varias,"5,5","1009,6",00,"1002,8",17
13120,2021-02-21,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"19,2","0,0","15,9",07:51,"22,5",Varias,99,"3,9","10,8",Varias,"9,6","1013,0",Varias,"1004,1",01
13121,2021-02-22,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"17,9","0,0","14,7",05:45,"21,1",12:25,12,"3,3","7,2",11:56,"5,3","1015,7",Varias,"1011,7",05
13122,2021-02-23,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"19,0","0,0","14,6",05:09,"23,4",13:39,99,"6,9","12,5",Varias,"9,5","1016,4",Varias,"1013,8",06


In [22]:
weather_grouped=df_weather.groupby(['fecha','system'],as_index=False)
df_weather_system=weather_grouped.mean(['tmed','prec','tmin','tmax','velmedia','sol','presMax','presMin'])
df_weather_system

,fecha,system,prec,tmin,tmax,velmedia,sol,presMax,presMin
0,2016-01-01,baleares,0.022222,8.544444,17.488889,2.366667,4.566561,1019.363378,1014.174425
1,2016-01-01,canarias,0.000000,14.280000,21.820000,2.871987,8.657036,1005.056728,1002.071764
2,2016-01-01,ceuta,0.000000,13.800000,21.200000,1.400000,7.600000,1015.400000,1012.400000
3,2016-01-01,melilla,0.000000,10.600000,20.400000,1.700000,9.100000,1019.900000,1016.600000
4,2016-01-02,baleares,0.177433,8.811111,18.344444,4.488889,5.233228,1017.352267,1012.785536
...,...,...,...,...,...,...,...,...,...
2923,2020-12-30,melilla,0.000000,8.600000,15.300000,2.500000,7.400000,1013.000000,1007.800000
2924,2020-12-31,baleares,0.700000,4.755556,13.988889,2.955556,7.744339,1013.191978,1006.900263
2925,2020-12-31,canarias,0.861503,14.330994,20.169522,4.216667,6.806768,999.909773,996.265359
2926,2020-12-31,ceuta,12.000000,8.000000,14.900000,2.800000,7.600000,1009.400000,1006.300000


## Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [ ]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('../Data/REGION_REE',header=0,index_col='Region')

# Me quedo solo con los distintos sistemas electricos existentes
region_system=region_ree[region_ree['geo_limit']!='ccaa']

region_system

In [ ]:
# Obtenemos los datos de REE a traves de su API. 

def lectura_ree_electric_system(d_inicio,d_fin,geo_id):
    '''Dada una fecha'''
    # meter esto en una funcion con su try-exception    
    # Dividir la lectura por años
    
    geo_limit=region_system[region_system['geo_id']==geo_id]['geo_limit']
    
    parametros={'start_date':date_ini,
            'end_date':date_end,
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':geo_limit[0],
            'geo_ids':geo_id}
    
    URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

    ree_gen=requests.get(URL_GEN,params=parametros)
    
    df_ree=pd.DataFrame()
    for i in range(20):
        try:
            df=pd.json_normalize(ree_gen.json()['included'][i]['attributes'],meta=['title','type'],record_path=['values'])
            df['system']=geo_limit[0]
            df_ree=df_ree.append(df)
        except:
            pass #Cuando no hay datos para mas tecnologías
    df_ree.reset_index(inplace=True,drop=True)
    
    return df_ree

In [ ]:
df_ree_system=pd.DataFrame()

In [ ]:

date_ini="2020-01-01T00:00:00UTC"
date_end="2020-12-31T00:00:00UTC"

for electric_system in tqdm(region_system['geo_id']):
    df_ree_system=df_ree_system.append(lectura_ree_electric_system(date_ini,date_end,electric_system))
df_ree_system 

# Limpieza de datos de REE

In [ ]:
# Renombro los campos
df_ree_system.rename(columns={'value':'Generacion_Mwh','title':'Tecnologia','type':'Renov_norenov'},inplace=True)

# Elimino la columna percentage por ser una columna generada de Generacion_Mwh.
df_ree_system.drop('percentage',axis=1,inplace=True)

# Cambio los valores nulos de Generacion en Mwh por 0
df_ree_system['Generacion_Mwh'].fillna(0)

# Elimino las filas para las cuales la fecha es nula y paso la fecha a formato corto.
df_ree_system['fecha']=df_ree_system['datetime'].str[:10]
df_ree_system=df_ree_system[~df_ree_system['fecha'].isna()]
df_ree_system.drop('datetime',axis=1,inplace=True)
df_ree_system

# Repositorio de información

Para evitar tener que leer todos los datos en cada ejecucion, guardamos los en formato csv desde los DataFrame de REE y AEMET

In [ ]:
#df_ree_system.to_csv('../Data/ree_system.csv')

In [ ]:
#df_weather_system.to_csv('../Data/weather.csv')

In [8]:
df_aux.to_csv('../Data/aux_weather.csv')

# Lectura desde el Repositorio

In [ ]:
df_ree_system=pd.read_csv('../Data/ree_system.csv',usecols=[1,2,3,4,5])
df_weather=pd.read_csv('../Data/weather.csv',index_col=0)

In [ ]:
import seaborn as sns

sns.pairplot(df_weather);

Vemos que en funcion de la presion (minima y maxima):
    - Hay 2 grupos muy claramente diferenciandos para temperaturas , velocidad del viento y horas de sol.
    - Y 2 grupos pero no tan claramente diferenciados para las precipitaciones
    - Se ve claramente un dato outlier para precipitaciones y otro para temperatura minima

# Outlier

Vamos a buscar los outlier de precipitaciones y temperatura minima, por si fuesen datos erroneos.

### Outlier de Precipitaciones

El 28-04-2017 tenemos un dato máximo en la serie histórica de precipitaciones con 184,4mm en la estación de CEUTA, revisando el dato vemos que ese dato si es correcto y así lo recoge la propia AEMET.

https://aemetblog.es/2017/06/21/resumen-climatico-de-la-primavera-2017/

In [ ]:
maxprec_mm=df_weather[df_weather['prec']==df_weather['prec'].max()]['prec']
maxprec_fec=df_weather[df_weather['prec']==df_weather['prec'].max()]['fecha']
plt.figure(figsize=(15,6))

plt.scatter(df_weather['fecha'],df_weather['prec'],alpha=0.5)
plt.scatter(maxprec_fec,maxprec_mm,c='r');


In [ ]:
df_weather[df_weather['prec']==df_weather['prec'].max()]

### Outlier de Temperatura mínima

El 02-08-2020 tenemos un dato máximo en la serie histórica de temperaturas minimas de 31,8mm en la estación de MELILLA, revisando el dato vemos que ese dato si es correcto y así lo recogen varios periodicos de esos días

https://elfarodemelilla.es/aviso-naranja-altas-temperaturas-melilla-registrara-hoy-28-grados-minima/

In [ ]:
tminGrouped=df_weather.groupby('fecha')['tmin'].mean().reset_index()
maxtmin_celsius=df_weather[df_weather['tmin']==df_weather['tmin'].max()]['tmin']
maxtmin_fec=df_weather[df_weather['tmin']==df_weather['tmin'].max()]['fecha']

In [ ]:

plt.figure(figsize=(15,6))
plt.plot(tminGrouped['fecha'],tminGrouped['tmin'],c='k');
plt.scatter(df_weather['fecha'],df_weather['tmin'],alpha=0.2);
plt.scatter(maxtmin_fec,maxtmin_celsius,c='r');


In [ ]:
df_weather[df_weather['tmin']==df_weather['tmin'].max()]

In [ ]:
df_weather

# Union of weather and generation data

## Features 

Para hacer un primer modelo quito generación total

In [ ]:
df_ree_system=df_ree_system[~(df_ree_system['Renov_norenov']=='Generación total')]
df_ree_system.reset_index(inplace=True,drop=True) # Reset index to merge de info after the features tratment


## Electric System Encoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder


enc=OneHotEncoder().fit(df_ree_system[['Tecnologia','Renov_norenov','system']])

df_transform=enc.transform(df_ree_system[['Tecnologia','Renov_norenov','system']])

column_names=enc.get_feature_names(['Tecnologia','Renov_norenov','system'])
df_onehot =  pd.DataFrame(df_transform.todense(), columns= column_names)

In [ ]:
df_onehot

# Encoder Weather

In [ ]:

enc_weather=OneHotEncoder().fit(df_weather[['electric_system']])

df_weather_transf=enc_weather.transform(df_weather[['electric_system']])

column_names=enc_weather.get_feature_names(['electric_system'])
df_weather_onehot =  pd.DataFrame(df_weather_transf.todense(), columns= column_names)

In [ ]:
df_weather_onehot

###  Tratamiento de fechas

Para pasar las fechas aun formato en que el salto entre el ultimo dia de un mes y el primer dia del siguiente sea continuo, uso los cosenos de los dias y meses. 

Para ello situo los valores de los 31 dias en angulos iguales calulandolos como $Dia(x)=\cos\frac{2\pi}{31}x$

Del mismo modo para los meses situo cada mes en $Mes(x)=\cos\frac{2\pi}{12}x$

In [ ]:
df_ree_system['year']=df_ree_system['fecha'].str.slice(0,4)
df_ree_system['day']=df_ree_system['fecha'].str.slice(8,10)
df_ree_system['month']=df_ree_system['fecha'].str.slice(5,7)

In [ ]:
df_ree_system['day_t']=np.cos(((2*np.pi)/31)*df_ree_system['day'].astype(int))
df_ree_system['month_t']=np.cos(((2*np.pi)/12)*df_ree_system['month'].astype(int))
df_ree_system

In [ ]:
df_ree_system.drop(['month','day','Tecnologia','Renov_norenov','system'],axis=1,inplace=True)

In [ ]:
df_weather['year']=df_weather['fecha'].str.slice(0,4)
df_weather['day']=df_weather['fecha'].str.slice(8,10)
df_weather['month']=df_weather['fecha'].str.slice(5,7)
df_weather['day_t']=np.cos((2*np.pi)/df_weather['day'].astype(int))
df_weather['month_t']=np.cos(((2*np.pi)/12)*df_weather['month'].astype(int))


In [ ]:
df_weather.drop(['day','month'],axis=1,inplace=True)

# Unir las 3 tablas df_weather, df_ree_system, df_onehot

In [ ]:
df_ree_features=pd.merge(df_ree_system,df_onehot, how='inner', left_index=True,right_index=True)
df_ree_features

In [ ]:

# df_tratado=pd.merge(df_ree_features,df_weather,how='inner', on='fecha')
# df_tratado.drop(['year_x','day_t_x','month_t_x'],axis=1,inplace=True)

In [ ]:


df_tratado_num=df_ree_features.select_dtypes(np.number)
df_tratado_num

# Models

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV

target='Generacion_Mwh'

y=df_tratado_num[target]
X=df_tratado_num.drop([target],axis=1)



reg_KN=GridSearchCV(KNeighborsRegressor(),param_grid={"n_neighbors":np.arange(3,15)},scoring='neg_mean_absolute_error')
reg_KN.fit(X,y)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

reg_DT=GridSearchCV(DecisionTreeRegressor(),param_grid={'max_depth':range(3,100)},scoring='neg_mean_absolute_error')
reg_DT.fit(X,y)

# Evaluation

Crear tabla de resultados.

In [ ]:
print(reg_KN.best_score_)
print(reg_KN.best_params_)

In [ ]:
print(reg_DT.best_score_)
print(reg_DT.best_params_)

# Interfaz y ploteado
